In [1]:
!pip install tensorflow-gpu==2.0.0-alpha0

    100% |████████████████████████████████| 332.1MB 55kB/s 
    100% |████████████████████████████████| 61kB 21.1MB/s 
    100% |████████████████████████████████| 419kB 10.3MB/s 
    100% |████████████████████████████████| 3.0MB 7.2MB/s 


In [1]:
import os
import tensorflow as tf
from google.colab import drive
import json
drive.mount('gdrive')

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


In [2]:
# only need to run once -- stores data repo on google drive
!git clone "https://github.com/LasseRegin/medical-question-answer-data" "gdrive/My Drive/Colab Notebooks/medical-question-answer-data"

fatal: destination path 'gdrive/My Drive/Colab Notebooks/medical-question-answer-data' already exists and is not an empty directory.


In [3]:
dataset_name = 'ehealthforumQAs'

with open('gdrive/My Drive/Colab Notebooks/medical-question-answer-data/%s.json'%dataset_name) as file:
  ds = json.load(file)
  file.close()
print('dataset length: ', len(ds))
ds[0]

dataset length:  171


{'answer': 'stopping smoking is about will power and being steadfast. you can stop safely by having bupropion or nicotine patch cover initially in consult with a doctor. contact an addiction clinic near you. wishing you best of health thanks',
 'question': 'how do i stop smoking now',
 'tags': ['addiction', 'stop smoking'],
 'url': 'http://ehealthforum.com/health/stop-smoking-question-t462882.html'}

In [4]:
examples = []
for sample in ds:
  question, answer, tags = sample['question'], sample['answer'], sample['tags']
  example = tf.train.Example()
  example.features.feature['question'].bytes_list.value.append(str.encode(question))
  example.features.feature['answer'].bytes_list.value.append(str.encode(answer))
  example.features.feature['tags'].bytes_list.value.extend([str.encode(tag) for tag in tags])
  examples.append(example.SerializeToString())
tf.train.Example.FromString(examples[0])

features {
  feature {
    key: "answer"
    value {
      bytes_list {
        value: "stopping smoking is about will power and being steadfast. you can stop safely by having bupropion or nicotine patch cover initially in consult with a doctor. contact an addiction clinic near you. wishing you best of health thanks"
      }
    }
  }
  feature {
    key: "question"
    value {
      bytes_list {
        value: "how do i stop smoking now"
      }
    }
  }
  feature {
    key: "tags"
    value {
      bytes_list {
        value: "addiction"
        value: "stop smoking"
      }
    }
  }
}

In [0]:
tfr = tf.data.Dataset.from_tensor_slices(examples)
writer = tf.data.experimental.TFRecordWriter('gdrive/My Drive/Colab Notebooks/medical-question-answer-data/%s.tfrecords'%dataset_name)
writer.write(tfr)